In [3]:
from pybiomart import Server, Dataset
import requests

server = Server(host='www.ensembl.org', use_cache = False)
mart = server['ENSEMBL_MART_ENSEMBL']
dataset = server.marts['ENSEMBL_MART_ENSEMBL'].datasets['drerio_gene_ensembl']

In [8]:
result = dataset.query(attributes=['external_gene_name', 'ensembl_gene_id'])
result.to_csv("zebrafish_gene_name_id_map.txt")

In [31]:
df_map = pd.read_csv("zebrafish_gene_name_id_map.txt")
gene_names = df_map["Gene name"].tolist()
gene_names_lower = [str(name).lower() for name in gene_names]
df_map["Gene name lower"] = gene_names_lower

df_tf = pd.read_csv("tf_zebrafish.csv")

for index, row in df_tf.iterrows():
    if row["ensembl_id"] == "MISSING":
        gene_name = row["Gene"]
        gene_id_rows = df_map[df_map["Gene name lower"] == gene_name.lower()]
        if gene_id_rows.empty:
            print("Could not find row for", gene_name)
            continue

        found_id = gene_id_rows.iloc[0]["Gene stable ID"]
        df_tf.iat[index, 19] = found_id
        print("Set", gene_name, "to", found_id)

print("hmm")
df_tf.to_csv("tf_zebrafish.csv", index=False)

Could not find row for iffo2a
Could not find row for -
Could not find row for ccdc50
Set cyb561a3 to ENSDARG00000028257
Could not find row for rasd2
Set igdcc4 to ENSDARG00000076919
Set rasgrf2 to ENSDARG00000002816
Could not find row for ano5a
Set tnfaip3 to ENSDARG00000027777
Could not find row for eurl
Could not find row for tjp1a
Could not find row for si:ch211-162e15.3
Set pxdn to ENSDARG00000098733
Set cdk2ap1 to ENSDARG00000115248
Could not find row for zmp:0000001316
Set guca1a to ENSDARG00000100747
Could not find row for afdna
Could not find row for si:ch211-189a21.1
Could not find row for ano8a
Set cemip to ENSDARG00000039881
Could not find row for ipmka
Could not find row for LOC561325
Could not find row for LOC571912
Could not find row for si:ch211-251j10.3
Could not find row for si:ch211-190p8.2
Could not find row for sept7a
Could not find row for im:7138239
Could not find row for mogat3a
Could not find row for ank3a
Could not find row for si:ch211-163l21.8
Set aars2 to EN

In [15]:
df_tf = pd.read_csv("tf_zebrafish.csv")
df_tf = df_tf.drop(columns=["Unnamed: 0.1","Unnamed: 0"])
df_tf.to_csv("tf_zebrafish.csv", index=False)


In [34]:
df_tf = pd.read_csv("tf_zebrafish.csv")
df_excel = pd.DataFrame(columns=["gene_name", "tf", "gene_id"])
for index, row in df_tf.iterrows():
    if row["ensembl_id"] == "MISSING":
        continue

    tfs = "["
    for col in df_tf.columns:
        if col == "Unnamed: 0" or col == "Gene" or col == "ensembl_id":
            continue

        tfs = tfs + str(row.loc[col]) + ", "

    tfs = tfs[:-2] + "]"
    df_excel.loc[len(df_excel)] = [row["Gene"], tfs, row["ensembl_id"]]

#df_excel.to_csv("tf_efxcel.csv", index=False)
df_excel.to_excel("tf_excel.xlsx", index=False)

In [2]:
import h5py
import os
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd


datadir = "..\\training\\zebrafish_training"

# Get truth data
testfile = h5py.File(os.path.join(datadir, 'zebrafish_test.hdf5'), 'r')
valfile = h5py.File(os.path.join(datadir, 'zebrafish_val.hdf5'), 'r')
trainfile = h5py.File(os.path.join(datadir, 'zebrafish_train.hdf5'), 'r')
y = np.concatenate((testfile['embryoLabels'][:], valfile['embryoLabels'][:], trainfile['embryoLabels'][:]), axis=None)
names = np.concatenate((testfile['geneName'][:], valfile['geneName'][:], trainfile['geneName'][:]), axis=None)

df_labels = pd.DataFrame({"gene_id": names, "label": y})
table = pa.Table.from_pandas(df_labels)
pq.write_table(table, 'zebrafish_expression_embryo.parquet')

In [2]:
import h5py
import os
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd


datadir = "..\\training"

# Get truth data
file = h5py.File(os.path.join(datadir, 'zebrafish.hdf5'), 'r')
#y = np.concatenate((testfile['embryoLabels'][:], valfile['embryoLabels'][:], trainfile['embryoLabels'][:]), axis=None)
y = file['label'][:]
#names = np.concatenate((testfile['geneName'][:], valfile['geneName'][:], trainfile['geneName'][:]), axis=None)
names = file['geneName'][:]

df_labels = pd.DataFrame({"gene_id": names, "label": y})
table = pa.Table.from_pandas(df_labels)
pq.write_table(table, 'zebrafish_expression_median.parquet')